# Work in progress (latest update 17.11.22) 

In [ ]:
#all_skip
from fastMONAI.vision_all import *
from IPython.display import clear_output

import torchio as tio
import SimpleITK as sitk

In [ ]:
path = Path('../data')

In [ ]:
def preprocess(o, reorder, resample): 
    org_metadata = {}
    org_size = None
    if reorder:
        transform = tio.ToCanonical()
        o = transform(o)
        org_size = o.shape[1:]
    
    if resample and not all(np.isclose(o.spacing, resample)):
            transform = tio.Resample(resample)
            o = transform(o)
            
    return o, org_size

In [ ]:
def pred_postprocess(pred_mask, avg_disc=10437, percentage=0.2): 
    small_objects = avg_disc*percentage    
    labeled_mask, ncomponents = label(pred_mask)
    labeled_mask = remove_small_objects(np.array(labeled_mask), min_size=small_objects)

    return torch.Tensor(np.where(labeled_mask>0, 1., 0.))

In [ ]:
def inference(learn_inf, reorder, resample, fn:str, save_path:(str,Path)=None): 
    '''
    Predict on new data using exported model
    '''         

    org_img = tio.ScalarImage(fn)
    
    input_img = org_img 
    input_img, org_size= preprocess(input_img, reorder, resample)    
    
    
    pred, *_ = learn_inf.predict(input_img.data);
    
    
    pred_mask = do_pad_or_crop(pred.float(), input_img.shape[1:], padding_mode=0, mask_name=None)
    input_img.set_data(pred_mask)
    
        
    transform = tio.Resize(org_size, image_interpolation='nearest', label_interpolation='nearest')#(org_img)
    input_img = transform(input_img)    

    pir_itk = sitk.DICOMOrient(input_img.as_sitk(), ('').join(org_img.orientation))
    pir_array = sitk.GetArrayFromImage(pir_itk).transpose()
    
    org_img.set_data(pir_array[None])
    
        
    if save_path is not None:
        save_fn = 'pred_' + Path(fn).parts[-1]
        save_fn = Path(save_path)/save_fn
        img.save(save_fn, )

    return org_img

In [ ]:
reorder, resample = load_variables(pkl_fn='models/vars.pkl')
reorder, resample

(True, [4.4, 0.78, 0.78])

In [ ]:
models = [load_learner(f'models/{i}_learner.pkl', cpu=True) for i in range(3)]

In [ ]:
fn = '../data/external_t2w_spine/img/Img_01.nii.gz'

In [ ]:
save_folder = Path('../data/external_t2w_spine/preds')
save_folder.mkdir(parents=True, exist_ok=True)

In [ ]:
mask_data = None
for idx, learner in enumerate(models):
    mask = inference(learner, reorder, resample, fn)
    if mask_data is None: 
        mask_data = mask.data
        
    else: 
        mask_data = mask_data + mask.data

    if idx == 2: 
        mask_data = mask_data/3
        mask_data = torch.where(mask_data>0.5, 1., 0.)
        mask.set_data(mask_data)
        mask.save(save_folder/'Img_01.nii.gz')